In [1]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
import yfinance as yf
import json
from openai import OpenAI


def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    ticker = ddg.run(f"Ticker symbol of {company_name}")
    print(f"Retrieved ticker: {ticker}")
    return ticker


def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yf.Ticker(ticker)
    income_statement = stock.financials.to_json()
    print(f"Retrieved income statement for {ticker}")
    return income_statement


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yf.Ticker(ticker)
    balance_sheet = stock.balance_sheet.to_json()
    print(f"Retrieved balance sheet for {ticker}")
    return balance_sheet


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yf.Ticker(ticker)
    stock_performance = stock.history(period="3mo").to_json()
    print(f"Retrieved stock performance for {ticker}")
    return stock_performance


functions_map = {
    "get_ticker": get_ticker,
    "get_income_statement": get_income_statement,
    "get_balance_sheet": get_balance_sheet,
    "get_daily_stock_performance": get_daily_stock_performance,
}

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company, returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e., AAPL), returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e., AAPL), returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e., AAPL), returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]

client = OpenAI()

assistant = client.beta.assistants.create(
    name="Investor Assistant v2.8",
    instructions="You help users perform research on publicly traded companies to decide if they should buy the stock or not.",
    tools=functions,
    model="gpt-4o",
)

thread = client.beta.threads.create()

message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I want to know if Extra Space Storage stock is a good buy.",
)

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)


def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outputs = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outputs,
    )

In [11]:
get_run(run.id, thread.id).status

'completed'

In [12]:
get_messages(thread.id)

user: I want to know if Extra Space Storage stock is a good buy.
assistant: ### Extra Space Storage (EXR) Summary Analysis

#### Financial Performance
1. **Income Statement Overview**:
   - **Revenue** (2023): $2.56 billion, up from $1.92 billion in the prior year.
   - **Net Income** (2023): $803.20 million, a 6.68% decrease from $860.69 million in 2022.
   - **Earnings per Share (EPS)** (2023): $4.74, down from $6.41 in 2022.
   - **Expenses** increased significantly in 2023, contributing to the lower net income.

2. **Balance Sheet Highlights**:
   - **Total Assets** (2023): $27.46 billion.
   - **Total Liabilities** (2023): $12.04 billion, with long-term debt at $10.57 billion.
   - **Stockholders' Equity** (2023): $14.39 billion, a substantial increase from $3.26 billion in 2022.
   - **Net Debt** (2023): $10.92 billion, indicating a highly leveraged position.

#### Stock Performance
1. **Recent Stock Prices**:
   - **Current Price**: Around $154.36 (as of the latest data).
   - *

In [7]:
get_tool_outputs(run.id, thread.id)

Calling function: get_income_statement with arg {"ticker": "EXR"}
Retrieved income statement for EXR
Calling function: get_balance_sheet with arg {"ticker": "EXR"}
Retrieved balance sheet for EXR
Calling function: get_daily_stock_performance with arg {"ticker": "EXR"}
Retrieved stock performance for EXR


[{'output': '{"1703980800000":{"Tax Effect Of Unusual Items":-1668300.0,"Tax Rate For Calcs":0.025,"Normalized EBITDA":1882618000.0,"Total Unusual Items":-66732000.0,"Total Unusual Items Excluding Goodwill":-66732000.0,"Net Income From Continuing Operation Net Minority Interest":803198000.0,"Reconciled Depreciation":506053000.0,"Reconciled Cost Of Revenue":670910000.0,"EBITDA":1815886000.0,"EBIT":1309833000.0,"Net Interest Income":-352964000.0,"Interest Expense":437821000.0,"Interest Income":84857000.0,"Normalized Income":868261700.0,"Net Income From Continuing And Discontinued Operation":803198000.0,"Total Expenses":1323371000.0,"Total Operating Income As Reported":1170141000.0,"Diluted Average Shares":169220882.0,"Basic Average Shares":169216989.0,"Diluted EPS":4.74,"Basic EPS":4.74,"Diluted NI Availto Com Stockholders":801968000.0,"Average Dilution Earnings":0.0,"Net Income Common Stockholders":801968000.0,"Otherunder Preferred Stock Dividend":1230000.0,"Net Income":803198000.0,"Min

In [8]:
submit_tool_outputs(run.id, thread.id)

Calling function: get_income_statement with arg {"ticker": "EXR"}
Retrieved income statement for EXR
Calling function: get_balance_sheet with arg {"ticker": "EXR"}
Retrieved balance sheet for EXR
Calling function: get_daily_stock_performance with arg {"ticker": "EXR"}
Retrieved stock performance for EXR


Run(id='run_rR6Zxmvr4TuZElATke52yOQG', assistant_id='asst_qiSr861RHod2C1SqeUKVtfkN', cancelled_at=None, completed_at=None, created_at=1720475573, expires_at=1720476173, failed_at=None, incomplete_details=None, instructions='You help users perform research on publicly traded companies to decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=1720475628, status='queued', thread_id='thread_XXaHY5ai6ncsQKGEfHIZhHoO', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company, returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}), type='function'), FunctionTool(function=FunctionDefinition(name='get_income_s